In [ ]:
import numpy as np
import pandas as pd
import math
from sklearn.model_selection import KFold, train_test_split, TimeSeriesSplit

In [ ]:
import os
import sys

root = os.path.split(os.getcwd())[0]
if root not in sys.path:
    sys.path.append(root)

In [ ]:
%load_ext autoreload
%autoreload 2
from strategy.Positions import AbstractPosition, UniV3Position, BiCurrencyPosition
from strategy.Portfolio import Portfolio
from strategy.Data import PoolDataUniV3
from strategy.Backtest import AbstractStrategy, Backtest
from strategy.History import PortfolioHistory
from strategy.Strategies import BiCurrencyActive, BiCurrencyPassive, UniV3Passive, UniV3Active
from strategy.Viewers import PotrfolioViewer
from strategy.CrossValidation import FolderSimple, CrossValidation

from strategy import  Pool, Token, Fee, Frequency

In [ ]:
def tick_to_price(tick, decimal_diff=10):
    price = np.power(1.0001, tick) / 10 ** decimal_diff
    return price

def price_to_tick(price, decimal_diff=10):
    tick = math.log(price, 1.0001) + decimal_diff * math.log(10, 1.0001)
    return int(round(tick))

In [ ]:
pool = Pool(Token.WBTC, Token.WETH, Fee.MIDDLE)

In [ ]:
data = PoolDataUniV3.from_folder(pool)
data.preprocess()

In [ ]:
swaps = data.swaps

In [ ]:
def tick_to_price(tick, decimal_diff=10):
    price = np.power(1.0001, tick) / 10 ** decimal_diff
    return price

def price_to_tick(price, decimal_diff=10):
    tick = math.log(price, 1.0001) + decimal_diff * math.log(10, 1.0001)
    return int(round(tick))

In [ ]:
mint_tolerance = 30
grid_width = 60
grid_num = 1
burn_num = 12

# burn_tolerance = grid_width * grid_num - mint_tolerance
burn_tolerance = grid_width * (grid_num + burn_num) - mint_tolerance

univ3_active = UniV3Active(mint_tolerance, burn_tolerance, grid_width, grid_num, 12, 18, pool, 0.01)

In [ ]:
# univ3_passive = UniV3Passive(12, 18, pool, 0.01)

In [ ]:
bi_curr_strat = BiCurrencyActive(2100, 12, 18, pool, 0.01)

In [ ]:
cv = CrossValidation(FolderSimple(), bi_curr_strat)

In [ ]:
folds_result = cv.backtest(data)

In [ ]:
cv_res = cv.aggregate(folds_result)

In [ ]:
cv_res.plot.bar()

In [ ]:
cv_res.mean()

In [ ]:
f1, f2, f3, f4 = PotrfolioViewer(folds_result['fold_5']).draw_portfolio()

In [ ]:
f2

In [ ]:
f4

In [ ]:
swaps['date'] = swaps.reset_index()['timestamp'].dt.normalize().to_numpy()

In [ ]:
swaps.groupby('date')['tick'].std().rolling(7).mean().plot()

In [ ]:
(swaps.groupby('date')['tick'].max() - swaps.groupby('date')['tick'].min()).rolling(7).mean().plot()